In [1]:
import json, re, pathlib, xml.etree.ElementTree as ET
from collections import defaultdict
from datetime import datetime

# <<< Pfade anpassen, falls nötig >>>
SLN_PATH   = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\TestProjektTwinCATEvents.sln"
OUT_XML    = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\io_mappings.xml"
OUT_JSON   = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\io_mappings.json"

# util: erstes n Zeichen eines Strings sicher anzeigen
def head(text, n=600):
    t = (text or "")[:n].replace("\r"," ").replace("\n"," ")
    return t + ("…" if text and len(text) > n else "")

# util: TwinCAT Pfad -> Teilstücke
def split_tc_path(p):
    return [s for s in (p or "").split("^") if s]

# util: VarA/VarB ("PlcTask Inputs^MAIN.bX" bzw. "Term 1 (EK1100)^...^Channel 1^Input")
def parse_var_side(var_str):
    parts = split_tc_path(var_str)
    return {
        "raw": var_str,
        "parts": parts,
        "is_plc_task": parts[0].lower().startswith("plctask "),
        "channel": parts[-1] if parts else ""
    }

# util: Adresse Byte.Bit aus „Channel X^Input“ ziehen wir später aus OwnerB-Pfad über Prozessabbild (Erweiterung möglich)



In [2]:
import win32com.client as com

dte = com.Dispatch("TcXaeShell.DTE.17.0")  # ggf. Version anpassen (17=VS 2022)
dte.SuppressUI = False
dte.MainWindow.Visible = True

solution = dte.Solution
solution.Open(SLN_PATH)

# .tsproj suchen
tc_project = None
print("Projects in Solution:")
for i in range(1, solution.Projects.Count + 1):
    p = solution.Projects.Item(i)
    print(f"  Index {i}: Name={p.Name}, FullName={p.FullName}")
    if p.FullName.lower().endswith(".tsproj"):
        tc_project = p

if tc_project is None:
    raise RuntimeError("Kein TwinCAT-Systemprojekt (.tsproj) in der Solution gefunden")

print("Verwende TwinCAT-Projekt:", tc_project.Name)
sys_mgr = tc_project.Object  # SystemManager (Automation Interface)


Projects in Solution:
  Index 1: Name=TestProjektTwinCATEvents, FullName=C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\TestProjektTwinCATEvents\TestProjektTwinCATEvents.tsproj
Verwende TwinCAT-Projekt: TestProjektTwinCATEvents


In [3]:
# 1) rohes Mapping-XML erzeugen (alle konfigurierten Links, u. a. PLC<->I/O)
xml_text = sys_mgr.ProduceMappingInfo()  # ITcSysManager3::ProduceMappingInfo
with open(OUT_XML, "w", encoding="utf-8") as f:
    f.write(xml_text or "")
print("Mapping-XML gespeichert:", OUT_XML)
print("XML-Head:", head(xml_text, 500))

# 2) XML -> strukturierte Liste von VarLinks
links = []
if xml_text and "<VarLinks" in xml_text:
    root = ET.fromstring(xml_text)
    # Struktur lt. Beckhoff-Doku:
    # <VarLinks>
    #   <OwnerA Name="TIPC^..."> <OwnerB Name="TIID^..."><Link VarA="..." VarB="..."/></OwnerB> ...
    #   <OwnerA Name="TIID^..."> <OwnerB Name="TIPC^..."><Link VarA="..." VarB="..."/></OwnerB> ...
    # </VarLinks>
    for ownerA in root.findall(".//OwnerA"):
        ownerA_name = ownerA.attrib.get("Name", "")
        for ownerB in ownerA.findall("./OwnerB"):
            ownerB_name = ownerB.attrib.get("Name", "")
            for link in ownerB.findall("./Link"):
                varA = link.attrib.get("VarA", "")
                varB = link.attrib.get("VarB", "")
                rec = {
                    "ownerA": ownerA_name,
                    "ownerB": ownerB_name,
                    "varA": varA,
                    "varB": varB,
                    "sideA": parse_var_side(varA),
                    "sideB": parse_var_side(varB),
                }
                # Normalisieren: PLC-Seite immer unter 'plc', I/O-Seite unter 'io'
                if rec["sideA"]["is_plc_task"]:
                    rec["plc"] = rec["sideA"]; rec["io"] = rec["sideB"]
                elif rec["sideB"]["is_plc_task"]:
                    rec["plc"] = rec["sideB"]; rec["io"] = rec["sideA"]
                else:
                    # selten: TcCOM<->I/O oder PLC<->TcCOM – wir lassen roh stehen
                    rec["plc"] = None; rec["io"] = None
                links.append(rec)

print("PLC↔I/O-Links gefunden:", sum(1 for r in links if r["plc"] and r["io"]))

with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(links, f, ensure_ascii=False, indent=2)
print("OK:", OUT_JSON, f"({len(links)} Einträge)")


Mapping-XML gespeichert: C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\io_mappings.xml
XML-Head: <?xml version="1.0"?><VarLinks/>
PLC↔I/O-Links gefunden: 0
OK: C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\io_mappings.json (0 Einträge)


In [4]:
# Zelle 4 (enriched, überschreibt io_mappings.json mit Adressfeldern)

import re, json
from pathlib import Path

# << gleiche OUT_JSON wie in Zelle 3 >>
# OUT_JSON = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\twincat2\TwinCAT\io_mappings.json"

def _full_channel_path(ownerB_name: str, varB: str) -> str:
    return f"{ownerB_name}^{varB}"

# --- Parser-Helfer für Channel-XML (ProduceXml) ---

_num = r"[-]?\d+"
def _get_int(xml: str, tag: str):
    m = re.search(fr"<{tag}[^>]*>\s*({_num})\s*</{tag}>", xml)
    return int(m.group(1)) if m else None

def _get_hex_attr(xml: str, tag: str):
    m = re.search(fr"<{tag}[^>]*Hex=\"#x([0-9A-Fa-f]+)\"", xml)
    return m.group(1) if m else None

def _parse_channel_meta(xml_text: str):
    # Standard-Felder, die Beckhoff im Channel-XML bereitstellt (je nach Klemme/Version)
    vsize    = _get_int(xml_text, "VarBitSize")
    vaddr    = _get_int(xml_text, "VarBitAddr")    # Bitadresse im Prozessabbild
    vinout   = _get_int(xml_text, "VarInOut")      # 0 = Input, 1 = Output
    ams      = _get_int(xml_text, "AmsPort")
    igrp     = _get_int(xml_text, "IndexGroup")
    ioff     = _get_int(xml_text, "IndexOffset")
    ln       = _get_int(xml_text, "Length")
    igrp_hex = _get_hex_attr(xml_text, "IndexGroup")
    ioff_hex = _get_hex_attr(xml_text, "IndexOffset")

    # Falls einige Geräte statt VarBitAddr getrennte Offsets liefern:
    kv = {}
    for m in re.finditer(r"<([A-Za-z0-9_]+)>\s*([0-9]+)\s*</\1>", xml_text):
        tag, val = m.group(1), int(m.group(2))
        if "Offs" in tag or "Offset" in tag or tag in ("ByteOffset", "BitOffset"):
            kv[tag] = val
    for m in re.finditer(r'Name="([^"]*?(?:Offs|Offset)[^"]*)"\s*>\s*([0-9]+)\s*<', xml_text):
        kv[m.group(1)] = int(m.group(2))

    # Byte/Bit bevorzugt aus VarBitAddr, sonst aus gefundenen Tags
    if isinstance(vaddr, int):
        byte_off = vaddr // 8
        bit_off  = vaddr % 8
    else:
        byte_off = (kv.get("InputOffsByte") or kv.get("OutputOffsByte")
                    or kv.get("ByteOffset") or kv.get("OffsByte"))
        bit_off  = (kv.get("InputOffsBit")  or kv.get("OutputOffsBit")
                    or kv.get("BitOffset")  or kv.get("OffsBit"))

    return {
        "varBitSize": vsize,
        "varBitAddr": vaddr,
        "varInOut":   vinout,
        "amsPort":    ams,
        "indexGroup": igrp,
        "indexOffset": ioff,
        "length":     ln,
        "indexGroupHex": igrp_hex,
        "indexOffsetHex": ioff_hex,
        "byte_offset": byte_off,
        "bit_offset":  bit_off,
        "rawOffsets":  kv
    }

def _dir_letter(plc_path_lower: str, var_inout: int, chan_spec: str) -> str:
    # Priorität: Channel-Suffix → PLC-Pfad → VarInOut
    if chan_spec.endswith("^Input"):  return "I"
    if chan_spec.endswith("^Output"): return "Q"
    if "plctask inputs"  in plc_path_lower: return "I"
    if "plctask outputs" in plc_path_lower: return "Q"
    if var_inout == 0: return "I"
    if var_inout == 1: return "Q"
    return "?"

def _plc_var_only(plc_side_var: str) -> str:
    parts = plc_side_var.split("^")
    return parts[-1] if parts else plc_side_var

# --------- Iterate Links aus Zelle 3, lesen Channel-XML und anreichern ----------

bundle = []
missing = 0

for rec in links:
    if not rec.get("plc") or not rec.get("io"):
        continue

    plc_var_path = rec["plc"]["raw"]                   # z. B. "PlcTask Outputs^GVL_MBS.MBS_Leuchte_Ofen"
    plc_var_name = _plc_var_only(plc_var_path)         # z. B. "GVL_MBS.MBS_Leuchte_Ofen"
    io_owner     = rec["ownerB"] if rec["ownerB"].startswith("TIID^") else rec["ownerA"]
    io_chan_spec = rec["io"]["raw"]                    # z. B. "Channel 2^Output"
    full_io_path = _full_channel_path(io_owner, io_chan_spec)

    # Standardwerte
    meta = {
        "varBitSize": None, "varBitAddr": None, "varInOut": None,
        "amsPort": None, "indexGroup": None, "indexOffset": None, "length": None,
        "indexGroupHex": None, "indexOffsetHex": None,
        "byte_offset": None, "bit_offset": None, "rawOffsets": {}
    }
    raw_xml = ""
    try:
        ch_item = sys_mgr.LookupTreeItem(full_io_path)
        try:
            ch_xml = ch_item.ProduceXml(0)
        except TypeError:
            ch_xml = ch_item.ProduceXml()
        raw_xml = ch_xml
        meta = _parse_channel_meta(ch_xml)
    except Exception as e:
        missing += 1
        raw_xml = f"ERROR: {e}"

    # %I/%Q Byte.Bit bilden (falls Byte/Bit bekannt)
    d_letter = _dir_letter(plc_var_path.lower(), meta.get("varInOut"), io_chan_spec)
    if isinstance(meta.get("byte_offset"), int) and isinstance(meta.get("bit_offset"), int):
        pi_addr = f"{d_letter} {meta['byte_offset']}.{meta['bit_offset']}"
    else:
        # falls nur VarBitAddr ohne Aufspaltung vorhanden war, erneut rechnen
        vaddr = meta.get("varBitAddr")
        if isinstance(vaddr, int):
            pi_addr = f"{d_letter} {vaddr//8}.{vaddr%8}"
            meta["byte_offset"] = vaddr//8
            meta["bit_offset"]  = vaddr%8
        else:
            pi_addr = None

    bundle.append({
        "plc_path":     plc_var_path,
        "plc_var":      plc_var_name,
        "device_path":  io_owner,
        "channel_label":io_chan_spec,
        "io_path":      full_io_path,
        "direction":    "Input" if d_letter=="I" else "Output" if d_letter=="Q" else "Unknown",
        "ea_address":   pi_addr,                  # z. B. "Q 77.0" / "I 39.0"
        "varBitAddr":   meta["varBitAddr"],       # Bitadresse im Prozessabbild
        "varBitSize":   meta["varBitSize"],
        "varInOut":     meta["varInOut"],         # 0=Input, 1=Output
        "byte_offset":  meta["byte_offset"],      # bevorzugt aus VarBitAddr berechnet
        "bit_offset":   meta["bit_offset"],
        "amsPort":      meta["amsPort"],
        "indexGroup":   meta["indexGroup"],
        "indexGroupHex":meta["indexGroupHex"],
        "indexOffset":  meta["indexOffset"],
        "indexOffsetHex":meta["indexOffsetHex"],
        "length":       meta["length"],
        "raw_offsets":  meta["rawOffsets"],
        "io_raw_xml":   raw_xml[:4000]            # kürzen, wenn du Platz sparen willst
    })

# --- Speichern: überschreibt die bestehende io_mappings.json mit den angereicherten Einträgen ---
Path(OUT_JSON).write_text(json.dumps(bundle, indent=2, ensure_ascii=False), encoding="utf-8")
print(f"OK: {OUT_JSON}  ({len(bundle)} Links, {sum(1 for x in bundle if x['byte_offset'] is not None)} mit Byte/Bit, {missing} ohne Channel-XML)")


OK: C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\io_mappings.json  (0 Links, 0 mit Byte/Bit, 0 ohne Channel-XML)
